In [51]:
import keras
from keras.models import Sequential
#from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution3D, MaxPooling3D
from keras.models import load_model

from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils, generic_utils
import cv2
import math
import os
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import json

In [2]:
model = load_model('my_model.h5')

In [76]:
# open the .txt file which have names of test videos
f = open(r"Your own path of .txt file which have names of test videos", "r")
temp = f.read()
videos = temp.split('\n')

# creating a dataframe having video names
test = pd.DataFrame()
test['video_name'] = videos
test = test[:]
print(test)

                     video_name
0  S012C003P007R001A041_rgb.avi


In [77]:
# storing the frames from test videos
Test_array = []
Time = []
Pred_of_Time = []
for i in tqdm(range(test.shape[0])):
#for i in range(1):
    test_image_temp = []
    count = 0
    videoFile = test['video_name'][i]
    cap = cv2.VideoCapture(r'Your own path of test video' + videoFile)   # capturing the video from the given path
    frameCount = cap.get(7) #frame counts
    x = math.floor(frameCount / 15)
    #print([frameCount,x])
    while(cap.isOpened()):
        frameId = cap.get(1) #current frame number
        frameTime = cap.get(0)
        ret, frame = cap.read()
        if (ret != True):
            break
        if (count == 15):
            break
        if (frameId == (x * count)):
            # storing the frames in a new folder named train_1
            frame=cv2.resize(frame,(112,112),interpolation=cv2.INTER_AREA)
            #filename =r'D:\2020Spring\636_project\data\test_1/' + videoFile +"_frame%d.jpg" % count;count+=1
            #cv2.imwrite(filename, frame)
            frame = image.img_to_array(frame)
            # appending the image to the train_image list
            test_image_temp.append(frame)
            #print(len(test_image_temp))
            count+=1
            Time.append(frameTime)
    cap.release()
    Test_array.append(np.array(test_image_temp,dtype='float16'))
Test_array_f = np.array(Test_array,dtype='float16')
Test_array_f = Test_array_f.astype('float32')
Test_array_f -= np.mean(Test_array_f)
Test_array_f /=np.max(Test_array_f)
#Test_array_f[0][:][:][:][:] = Test_array[:][:][:][:]
#print(Test_array_f.shape)
#make prediction
prediction = model.predict(Test_array_f)
#print(prediction)
timelabel = np.zeros((15,2))
for i in range(len(Time)):
    Pred_of_Time.append(prediction[0][1])
    Time[i] = math.floor(Time[i])
    Time[i] /= 1000
plt.plot(Time, Pred_of_Time)
    timelabel[i][0] = Time[i]
    timelabel[i][1] = Pred_of_Time[i]
timelabel_l = timelabel.tolist()
print(timelabel_l)
with open('Cough.json', 'w') as f:
    json.dump(timelabel_l, f)

100%|██████████| 1/1 [00:00<00:00,  1.79it/s]


[[0.0, 0.9985839128494263], [0.133, 0.9985839128494263], [0.3, 0.9985839128494263], [0.466, 0.9985839128494263], [0.633, 0.9985839128494263], [0.8, 0.9985839128494263], [0.966, 0.9985839128494263], [1.133, 0.9985839128494263], [1.3, 0.9985839128494263], [1.466, 0.9985839128494263], [1.633, 0.9985839128494263], [1.8, 0.9985839128494263], [1.966, 0.9985839128494263], [2.133, 0.9985839128494263], [2.3, 0.9985839128494263]]
